## Pseudo-spectral algorithm for solving the non-dimensionalised version of the partial differential equations described in the paper "Evidence for a scale-dependent root-augmentation feedback and patchy invasion"

The dimensional model equations are given by equations 7 (see paper). The first two equations describe the rate of change of above-ground biomass of two species on the Brazilian Island of Trindade. Our model is able to qualitatively describe the ongoing invasion process on Trindade Island; specifically, $\textit{Guilandina bonduc}$ is spreading and displacing $\textit{Cyperus atlanticus Hemsl.}$ in a patchy invasion process. This is due to water limitation, but see the paper for more details on the underlying ecological processes. 

The non-dimensionalised form of the equations solved in this notebook are:

\begin{align}
\frac{\partial b_i}{\partial t} &= g^i_b b_i (1-b_i) - \mu_i b_i + \delta_{b_i} \frac{w}{w+w_i^*}\phi_i * b_i,~~ i=1,2,  \\
\frac{\partial w}{\partial t} &= p - \nu \left(1 - \sum_{i=1}^{n} \rho_i b_i \right) w - w \sum_{i=1}^{n} g_w^i + \delta_w \Delta w,
\end{align}

where 

\begin{align}
g_{b}^i &= \nu \lambda_i \int_\Omega g_i(\mathbf{x},\mathbf{x}',t) w(\mathbf{x}',t) d\mathbf{x}', \\
g_{w}^i &= \gamma_i \int_\Omega g_i(\mathbf{x}',\mathbf{x},t) b(\mathbf{x}',t) d\mathbf{x}',
\end{align}

and

\begin{align}
g_{i} &= \frac{1}{2 \pi \sigma_{g_i}^2} \operatorname{exp}\left(-\frac{|\mathbf{X}-\mathbf{X}'|^2}{2 [\sigma_{g_i}(1+b_i(\mathbf{X},T)]^2)}\right), //
\phi_{i} &= \frac{1}{2 \pi} \left[ \frac{\sigma_{d_i}}{\left(|\mathbf{x}-\mathbf{x}'|+\sigma_{d_i}^2 \right)^{3/2}} \right].
\end{align}

$b_1(\mathbf{x},t)$ and $b_2(\mathbf{x},t)$ represent $\textit{Guilandina}$ and $\textit{Cyperus}$, respectively, and $w$ represents the soil water concentration. The dimensional parameters in the paper can be related to the presented non-dimensional parameters via the following scalings:

| **Quantity** | **Scaling** | 
| --- | --- |
|$$b_{i}$$ | $$B_i/K_i$$ |
|$$w$$ | $$\Lambda_1 W/N $$ |
|$$h$$ | $$\Lambda_1 H/N$$ |
|$$\nu$$ | $$N/M_1$$ |
|$$\lambda_i$$ | $$\Lambda_i/\Lambda_1$$ |
|$$\mu_i$$ | $$M_i/M_1$$ |
|$$\alpha$$ | $$A/M_1$$ |
|$$q$$ | $$Q/K_1$$ |
|$$\mathbf{x}$$ | $$\mathbf{X}/S_{G_1}$$ |
|$$t$$ | $$M_1 T$$ |
|$$p$$ | $$\Lambda_1 P /(NM_1)$$ |
|$$\gamma_i$$ | $$\Gamma_i K_i /M_1$$ |
|$$\eta_i$$ | $$E_i K_i$$ |
|$$\rho_i$$ | $$R_i$$ |
|$$\sigma_{g_i}$$ | $$S_{G_i}/S_{G_1}$$ |
|$$\sigma_{\delta_i}$$ | $$S_{D_i}/S_{G_1}$$ | 
|$$\delta_{b_i}$$ | $$\Theta_i/M_i$$ |
|$$\delta_w$$ | $$D_W/(M_1S_{G_1}^2)$$ |
|$$w_i^*$$ | $$\Lambda_1 W_i^* /N$$ | 

**Requirements** - the following implementation requires installation of CuPy (https://docs.cupy.dev/en/stable/install.html), a prerequisite of which is an NVIDIA CUDA GPU with compute capability greater than 3.0. 

First, we import some modules, define the parameters that capture patchy invasion dynamics, and calculate the non-dimensional quantities described in the above table.

In [9]:
import numpy as np
import cupy as cp
from cupy.fft import fft2,ifft2
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.integrate import odeint
from gbgw import initsigmac, gfunc
from tqdm import tqdm

P=1300 # precipitation
E1=18; E2=5 # root-shoot ratio
K1=0.7; K2=0.35 # carrying capacity
M1=7; M2=7 # death rate
N=15; # evaporation rate
Lam1=0.06; Lam2=0.16 # growth rates
Gam1=15; Gam2=5 # water uptake rates
Rh1=0.1; Rh2=0.1 # evaporation reduction due to shading
SG1=0.5; # minimum lateral extent of roots
SD1=0.5; SD2=0.01; # dispersal distance
R1=0.25; R2=0.25; # maximum 
f1=0.05; f2=0.05; # fecundity
g1=250; g2=250; # seed-plant conversion factor
Om1=0.5; Om2=2 # 
DW=0.5; # soil water diffusion rate 

p=Lam1*P/(N*M1);
nu=N/M1; lam1=1; lam2=Lam2/Lam1; mu1=(M1+f1)/M1; mu2=(M2+f2)/M1;
gam1=Gam1*K1/M1; gam2=Gam2*K2/M1
eta1=E1*K1; eta2=E2*K2; rho1=Rh1; rho2=Rh2
sg1=1; sd1=SD1/SG1; sd2=SD2/SG1;
om1=Lam1*Om1/N; om2=Lam1*Om2/N
db1=R1*f1*g1/M1; db2=R2*f2*g2/M1;
dw=DW/(M1*SG1**2)

In [10]:
Xmax=100 # domain size
xmax=Xmax/SG1 # scaled domain
Lx=xmax/2 # for spectral domain
Ly=xmax/2 # for spectral domain
dt=0.02 # timestep
NN=128**10; # number of points in spatial discretisation
tmax=1000 # maximum time of solution
steps=int(tmax/dt) # number of timesteps             

dx=(2*Lx/NN)
x=(2*Lx/NN)*np.arange(-NN/2,NN/2,1); y=x; Ymax=Xmax
extent=[0,Xmax,0,Ymax] # for plotting
X=x*SG1; Y=X
[xx,yy]=np.asarray(np.meshgrid(x,y))

##############################################################################
#########################  Calculate steady state  ###########################
##############################################################################  
def model_no_space(y,t): # define spatially indepentent model
    u1,u2,v1=y
    du1dt=-mu1*u1+nu*u1*v1*(1-u1)*(1+eta1*u1)**2+db1*v1/(v1+om1)*u1
    du2dt=-mu2*u2+nu*lam2*u2*v1*(1-u2)*(1+eta2*u2)**2+db2*v1/(v1+om2)*u2
    dv1dt=p-nu*v1*(1-rho1*u1-rho2*u2)-v1*gam2*u2*(1+eta2*u2)**2 
    - v1*gam1*u1*(1+eta1*u1)**2
    dydt=[du1dt,du2dt,dv1dt]
    return dydt
tstop=500
tode=np.arange(0,tstop,0.0001)

b1p1, b2p1, wp1 = odeint(model_no_space, [1,0,0.05], tode)[-1] # pure b1 state
b1p2, b2p2, wp2 = odeint(model_no_space, [0,1,0.05], tode)[-1] # pure b2 state

##############################################################################
###########################  Initial conditions  #############################
##############################################################################

#  Pure b2 ICs with small noise term
u1 = np.ones([NN,NN])*b1p2 + np.random.uniform(0,0.01,(NN,NN))*b1p2
u2 = np.ones([NN,NN])*b2p2 + np.random.uniform(0,0.01,(NN,NN))*b2p2
v1 = np.ones([NN,NN])*wp2 + np.random.uniform(0,0.01,(NN,NN))*wp2

# Define gaussian function
def gauss2d(xx,yy,x0,y0,sig0):
    g=np.exp(-((xx-x0)**2+(yy-y0)**2)/(2*sig0**2))
    return g
# Add single small patch of b1 at location (x0, y0) with s.d. sig0 
x0=0; y0=0
sig0=1

u1=u1+(b1p1-b1p2)*gauss2d(xx,yy,x0,y0,sig0)
u2=u2+(b2p1-b2p2)*gauss2d(xx,yy,x0,y0,sig0)
v1=v1+(wp1-wp2)*gauss2d(xx,yy,x0,y0,sig0)

##############################################################################
########################  Construct spectral domain  #########################
##############################################################################

kx = (np.pi/Lx)*np.concatenate((np.arange(0,NN/2 +1),np.arange(-NN/2 +1,0)))
ky = (np.pi/Ly)*np.concatenate((np.arange(0,NN/2 +1),np.arange(-NN/2 +1,0)))
[kxx, kyy] = np.meshgrid(kx, ky)
ksq = kxx**2 + kyy**2

ksq = cp.asarray(ksq).astype('float32')
kxx = cp.asarray(kxx).astype('float32')

u1 = cp.array(u1).astype('float32')
u2 = cp.array(u2).astype('float32')
v1 = cp.array(v1).astype('float32')

u1hat = fft2(u1)
u2hat = fft2(u2)
v1hat = fft2(v1)

##############################################################################
#################### Define equations as cuda kernels  #######################
##############################################################################  

@cp.fuse()
def kin(u1, v1, u2, p):
    f1 = -mu1*u1
    f2 = -mu2*u2+nu*lam2*u2*v1*(1-u2)*(1+eta2*u2)**2
    g1 = p - nu*v1*(1-rho1*u1-rho2*u2) - v1*gam2*u2*(1+eta2*u2)**2
    return f1,f2,g1

@cp.fuse()
def ku1in(u1, v1, u2, p, gb1, sek1):
    ku1in = kin(u1, v1, u2, p)[0] \
    + nu*gb1*u1*(1-u1) \
    + db1*sek1*v1/(v1+om1)
    return ku1in

@cp.fuse()
def ku2in(u1, v1, u2, p, sek2):
    ku2in = kin(u1, v1, u2, p)[1] + db2*sek2*v1/(v1+om2)
    return ku2in

@cp.fuse()
def kv1in(u1,v1,u2,p,gw1):
    kv1in=kin(u1,v1,u2,p)[2]-gam1*gw1*v1 
    return kv1in

##############################################################################
#################### Construct integral decomposition ########################
##############################################################################  
Nl=5 # number of convolutions for integral approximation
smin1=1; smax1=(1+eta1);
phi1c=(1+eta1*u1);
philc=initsigmac(smin1,smax1,Nl)
Hfc=cp.zeros((Nl,NN,NN)).astype('float32')
for l in range(0,Nl):
    Hfc[l,:,:]=philc[l]**2*cp.exp(-philc[l]**2*ksq/2)/sg1**2
gb1, gw1 = gfunc(u1,v1,Hfc,phi1c,philc,NN,NN,Nl)
# Note: see paper of Gilad and Von Hardenberg (2006)

np.savez_compressed('data/0.0',
                    [u1.get(),v1.get(),u2.get(),p,0]) # save initial condition

/home/jamie/anaconda3/envs/phys/lib/python3.9/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


In [12]:
%matplotlib qt
maximum1=K1; maximum2=K2; minimum=0
fig1 = plt.gcf()
fig1.show()
fig1.canvas.draw()
ax1 = fig1.add_subplot(111)
im1 = ax1.imshow(u1.get(),
                 extent=extent,
                 cmap=plt.cm.Reds,
                 origin='lower',
                 interpolation='nearest',
                 alpha=1)
im2 = ax1.imshow(u2.get(),
                 extent=extent,
                 cmap=plt.cm.Blues,
                 origin='lower',
                 interpolation='nearest',
                 alpha=0.4)
divider = make_axes_locatable(ax1)
divider = make_axes_locatable(ax1)
fig1.suptitle('T='+str(int(0/M1))+' [yr]'+', \
              P='+str(int(np.round(p*N*M1/Lam1)))+' [mm]')
plt.pause(0.25) 

store_every = int(np.round(M1*0.2/dt)) # save every store_every time steps
plot_every = int(np.round(M1*0.1/dt)) # plot every plot_every time steps

In [13]:
for n in tqdm(range(0,steps)):
    phi1c=(1+eta1*u1);
    gb1,gw1=gfunc(u1,v1,Hfc,phi1c,philc,NN,NN,Nl) 
    sek1=cp.real(ifft2(cp.exp(-cp.abs(cp.sqrt(ksq))*sd1)*u1hat))
    ku11 = dt*fft2(ku1in(u1,v1,u2,p,gb1,sek1))
    del sek1,gb1
    sek2=cp.real(ifft2(cp.exp(-cp.abs(cp.sqrt(ksq))*sd2)*u2hat))
    ku21 =dt*fft2(ku2in(u1,v1,u2,p,sek2))
    del sek2
    kv11 =dt*fft2(kv1in(u1,v1,u2,p,gw1))
    del u1,v1,u2,gw1
    Eu1=1 # no diffusion term in u1 equation
    u12=cp.real(ifft2(Eu1*(u1hat+ku11/2)))
    u1=Eu1**2*u1hat+(Eu1**2*ku11)/6
    del Eu1,ku11
    Ev1=cp.exp(-(dw*ksq-0*1j*kxx)*dt/2)
    v12=cp.real(ifft2(Ev1*(v1hat+kv11/2)))
    v1=Ev1**2*v1hat+(Ev1**2*kv11)/6
    del Ev1,kv11
    Eu2=1 # no diffusion term in u2 equation
    u22=cp.real(ifft2(Eu2*(u2hat+ku21/2)))
    u2=Eu2**2*u2hat+(Eu2**2*ku21)/6
    del Eu2,ku21
       
    phi1c=(1+eta1*u12);
    gb1,gw1=gfunc(u12,v12,Hfc,phi1c,philc,NN,NN,Nl) 
    sek1=cp.real(ifft2(cp.exp(-cp.abs(cp.sqrt(ksq))*sd1)*fft2(u12)))
    ku12 =dt*fft2(ku1in(u12,v12,u22,p,gb1,sek1))
    del sek1,gb1
    sek2=cp.real(ifft2(cp.exp(-cp.abs(cp.sqrt(ksq))*sd2)*fft2(u22)))
    ku22 =dt*fft2(ku2in(u12,v12,u22,p,sek2))
    del sek2
    kv12 =dt*fft2(kv1in(u12,v12,u22,p,gw1))
    del u12,v12,u22,gw1
    Eu1=1
    u13=cp.real(ifft2(Eu1*u1hat+ku12/2))
    u1=u1+(2*Eu1*ku12)/6
    del Eu1,ku12    
    Ev1=cp.exp(-(dw*ksq-0*1j*kxx)*dt/2)
    v13=cp.real(ifft2(Ev1*v1hat+kv12/2))
    v1=v1+(2*Ev1*kv12)/6
    del Ev1,kv12
    Eu2=1
    u23=cp.real(ifft2(Eu2*u2hat+ku22/2))
    u2=u2+(2*Eu2*ku22)/6
    del Eu2,ku22
       
    phi1c=(1+eta1*u13);
    gb1,gw1=gfunc(u13,v13,Hfc,phi1c,philc,NN,NN,Nl) 
    sek1=cp.real(ifft2(cp.exp(-cp.abs(cp.sqrt(ksq))*sd1)*fft2(u13)))
    ku13 =dt*fft2(ku1in(u13,v13,u23,p,gb1,sek1))
    del sek1,gb1
    sek2=cp.real(ifft2(cp.exp(-cp.abs(cp.sqrt(ksq))*sd2)*fft2(u23)))
    ku23 =dt*fft2(ku2in(u13,v13,u23,p,sek2))
    del sek2
    kv13 =dt*fft2(kv1in(u13,v13,u23,p,gw1))
    del u13,v13,u23,gw1
    Eu1=1
    u14 = cp.real(ifft2(Eu1**2*u1hat+Eu1*ku13))
    u1=u1+(2*Eu1*ku13)/6
    del Eu1,ku13
    Ev1=cp.exp(-(dw*ksq-0*1j*kxx)*dt/2)
    v14 = cp.real(ifft2(Ev1**2*v1hat+Ev1*kv13))
    v1=v1+(2*Ev1*kv13)/6
    del Ev1,kv13
    Eu2=1
    u24 = cp.real(ifft2(Eu2**2*u2hat+Eu2*ku23))
    u2=u2+(2*Eu2*ku23)/6
    del Eu2,ku23
       
    phi1c=(1+eta1*u14);
    gb1,gw1=gfunc(u14,v14,Hfc,phi1c,philc,NN,NN,Nl) 
    sek1=cp.real(ifft2(cp.exp(-cp.abs(cp.sqrt(ksq))*sd1)*fft2(u14)))
    ku14 =dt*fft2(ku1in(u14,v14,u24,p,gb1,sek1))
    u1=cp.real(ifft2(u1+ku14/6))
    del sek1,gb1
    sek2=cp.real(ifft2(cp.exp(-cp.abs(cp.sqrt(ksq))*sd2)*fft2(u24)))
    ku24 =dt*fft2(ku2in(u14,v14,u24,p,sek2))
    u2=cp.real(ifft2(u2+ku24/6))
    del sek2,ku24
    kv14 =dt*fft2(kv1in(u14,v14,u24,p,gw1))
    v1=cp.real(ifft2(v1+kv14/6))
    del kv14,gw1
    u1hat=fft2(u1) # not sure why but this seems to stabilise the method
    v1hat=fft2(v1) # perhaps something to do with numpys fft2
    u2hat=fft2(u2) # not needed in the equivalent 1D numerical scheme...
    t=(n+1)*dt

    if (int(n+1)  % store_every) == 0:
        np.savez_compressed('data/'+str(np.round(t,1))
                            , [u1.get(), v1.get(), u2.get(), p, t])
    if (n % plot_every) == 0:
        im1.set_array(u1.get())
        im2.set_array(u2.get())
        fig1.suptitle('T='+str(int(t/M1))+' [yr]'+',  P='+str(int(np.round(p*N*M1/Lam1)))+' [mm]')
        fig1.canvas.draw()
        plt.pause(0.01) 

 14%|█████▎                                | 6965/50000 [03:42<22:55, 31.28it/s]


KeyboardInterrupt: 